In [1]:
import numpy as np
import scipy.io
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy.signal import butter, lfilter
from scipy.signal import butter, sosfilt, sosfreqz
from scipy.signal import welch
from scipy import signal
import os
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [21]:
#save_path = 'D:\EECE499\Features\GSRFeatures.xlsx'
save_path = '..\..\..\GSRFeatures.xlsx'

In [4]:
sample_rate = 128

In [5]:
def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y

def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=6):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [6]:
max_length = 17215

In [7]:
yup = np.array(list(range(1, 17216)))

In [8]:
yup

array([    1,     2,     3, ..., 17213, 17214, 17215])

In [9]:
17216/2460

6.998373983739837

In [10]:
window = 2460
x = [2460, 4920, 7380, 9840, 12300, 14760]
y = [1230, '-', 3690, '-', 7380, '-', 9840, '-', 12300]

In [11]:
window_intervals_even = np.array([1, 2, 3, 4, 5, 6])*window
window_intervals_odd = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5])*window

In [12]:
window_intervals_even

array([ 2460,  4920,  7380,  9840, 12300, 14760])

In [13]:
window_intervals_odd = window_intervals_odd.astype(int)
window_intervals_odd

array([ 1230,  3690,  6150,  8610, 11070, 13530, 15990])

In [14]:
for i in range(1, 6):
    print(yup[window_intervals_even[i - 1]:window_intervals_even[i]].shape[0] == 2460)
    
for i in range(1, 7):
    print(yup[window_intervals_odd[i - 1]:window_intervals_odd[i]].shape[0] == 2460)

True
True
True
True
True
True
True
True
True
True
True


In [17]:
data_frames = []

for user_id in range(1, 59):
    print('User ID: ' + str(user_id))
    
    mean_skin_resistance = [[] for i in range(13)]
    mean_first_derivative_res = [[] for i in range(13)]
    mean_abs_first_derivative_res = [[] for i in range(13)]
    mean_neg_first_derivative_res = [[] for i in range(13)]
    percentage_neg_first_derivative_res = [[] for i in range(13)]
    std_skin_resistance = [[] for i in range(13)]
    avg_minima_con = [[] for i in range(13)]
    #avg_rising_time_res =[]
    #log_power_density = []
    std_skin_conductance = [[] for i in range(13)]
    mean_first_derivative_con = [[] for i in range(13)]
    mean_abs_first_derivative_con = [[] for i in range(13)]
    mean_abs_second_derivative_con = [[] for i in range(13)]
    avg_minima_res = [[] for i in range(13)]
    #log_power_density
    psd_subband_01 = []
    psd_subband_02 = []
    psd_subband_03 = []
    psd_subband_04 = []
    zero_crossing_con_slow = [[] for i in range(13)]
    zero_crossing_con_very_slow = [[] for i in range(13)]
    psd_subband_11 = []
    psd_subband_12 = []
    psd_subband_13 = []
    psd_subband_14 = []
    psd_subband_15 = []
    psd_subband_16 = []
    psd_subband_17 = []
    psd_subband_18 = []
    psd_subband_19 = []
    psd_subband_10 = []
    
    presentation_id = []
    
    for clip_id in range(1, 37):
        #print('Clip ID: ' + str(clip_id))
        
        #data_path = 'D:/EECE499/Raw/MyECGFunc/ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'    
        data_path = './../../ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'
        
        presentation_id.append(str(user_id).zfill(2) + str(clip_id).zfill(2))
        
        if os.path.isfile(data_path):
            
            signal = scipy.io.loadmat(data_path)
            resistance = signal['Data_GSR'][:,4]
            time_stamp = signal['Data_GSR'][:,0]
            
            resistance = np.array(resistance)
            resistance = savgol_filter(resistance, 501, 3)
            conductance = 1 / resistance
            if (max_length > resistance.shape[0]):
                resistance = np.pad(resistance, (0, max_length - resistance.shape[0]), 'mean')
                conductance = np.pad(conductance, (0, max_length - conductance.shape[0]), 'mean')
            else:
                resistance = resistance[:17215]
                conductance = conductance[:17215]
            #print(resistance.shape)
            
            #plt.plot(resistance)
            #plt.show()
            
            #print(mean_skin_resistance)
            
            mean_skin_resistance[0].append(np.mean(resistance[:window_intervals_even[0]]))
            for i in range(1, 6):
                mean_skin_resistance[i].append(np.mean(resistance[window_intervals_even[i - 1]:window_intervals_even[i]]))
            mean_skin_resistance[6].append(np.mean(resistance[window_intervals_even[5]:]))
            for i in range(1, 7):
                mean_skin_resistance[i + 6].append(np.mean(resistance[window_intervals_odd[i - 1]:window_intervals_odd[i]]))
                
            
            del_t = time_stamp[1] - time_stamp[0]
            
            first_derivatives = np.diff(resistance[:window_intervals_even[0]]) / del_t
            mean_first_derivative_res[0].append(np.mean(first_derivatives))
            for i in range(1, 6):
                first_derivatives = np.diff(resistance[window_intervals_even[i - 1]:window_intervals_even[i]]) / del_t
                mean_first_derivative_res[i].append(np.mean(first_derivatives))
            first_derivatives = np.diff(resistance[window_intervals_even[5]:]) / del_t
            mean_first_derivative_res[6].append(np.mean(first_derivatives))
            for i in range(1, 7):
                first_derivatives = np.diff(resistance[window_intervals_odd[i - 1]:window_intervals_odd[i]]) / del_t
                mean_first_derivative_res[i + 6].append(np.mean(first_derivatives))
                
            
            first_derivatives = np.diff(resistance[:window_intervals_even[0]]) / del_t
            mean_abs_first_derivative_res[0].append(np.mean(np.absolute(first_derivatives)))
            for i in range(1, 6):
                first_derivatives = np.diff(resistance[window_intervals_even[i - 1]:window_intervals_even[i]]) / del_t
                mean_abs_first_derivative_res[i].append(np.mean(np.absolute(first_derivatives)))
            first_derivatives = np.diff(resistance[window_intervals_even[5]:]) / del_t
            mean_abs_first_derivative_res[6].append(np.mean(np.absolute(first_derivatives)))
            for i in range(1, 7):
                first_derivatives = np.diff(resistance[window_intervals_odd[i - 1]:window_intervals_odd[i]]) / del_t
                mean_abs_first_derivative_res[i + 6].append(np.mean(np.absolute(first_derivatives)))
                
            
            first_derivatives = np.diff(resistance[:window_intervals_even[0]]) / del_t
            mean_neg_first_derivative_res[0].append(np.mean(first_derivatives[first_derivatives < 0]))
            for i in range(1, 6):
                first_derivatives = np.diff(resistance[window_intervals_even[i - 1]:window_intervals_even[i]]) / del_t
                mean_neg_first_derivative_res[i].append(np.mean(first_derivatives[first_derivatives < 0]))
            first_derivatives = np.diff(resistance[window_intervals_even[5]:]) / del_t
            mean_neg_first_derivative_res[6].append(np.mean(first_derivatives[first_derivatives < 0]))
            for i in range(1, 7):
                first_derivatives = np.diff(resistance[window_intervals_odd[i - 1]:window_intervals_odd[i]]) / del_t
                mean_neg_first_derivative_res[i + 6].append(np.mean(first_derivatives[first_derivatives < 0]))
                
            
            first_derivatives = np.diff(resistance[:window_intervals_even[0]]) / del_t
            percentage_neg_first_derivative_res[0].append(np.mean(((first_derivatives[first_derivatives < 0]).sum() / len(first_derivatives))))
            for i in range(1, 6):
                first_derivatives = np.diff(resistance[window_intervals_even[i - 1]:window_intervals_even[i]]) / del_t
                percentage_neg_first_derivative_res[i].append(np.mean(((first_derivatives[first_derivatives < 0]).sum() / len(first_derivatives))))
            first_derivatives = np.diff(resistance[window_intervals_even[5]:]) / del_t
            percentage_neg_first_derivative_res[6].append(np.mean(((first_derivatives[first_derivatives < 0]).sum() / len(first_derivatives))))
            for i in range(1, 7):
                first_derivatives = np.diff(resistance[window_intervals_odd[i - 1]:window_intervals_odd[i]]) / del_t
                percentage_neg_first_derivative_res[i + 6].append(np.mean(((first_derivatives[first_derivatives < 0]).sum() / len(first_derivatives))))  
                
            
            std_skin_resistance[0].append(np.std(resistance[:window_intervals_even[0]]))
            for i in range(1, 6):
                std_skin_resistance[i].append(np.std(resistance[window_intervals_even[i - 1]:window_intervals_even[i]]))
            std_skin_resistance[6].append(np.std(resistance[window_intervals_even[5]:]))
            for i in range(1, 7):
                std_skin_resistance[i + 6].append(np.std(resistance[window_intervals_odd[i - 1]:window_intervals_odd[i]]))
                
            
            peaks, _ = find_peaks(resistance[:window_intervals_even[0]])
            avg_minima_con[0].append(len(peaks) / 2460)
            for i in range(1, 6):
                peaks, _ = find_peaks(resistance[window_intervals_even[i - 1]:window_intervals_even[i]]) 
                avg_minima_con[i].append(len(peaks) / 2460)
            peaks, _ = find_peaks(resistance[window_intervals_even[5]:]) 
            avg_minima_con[6].append(len(peaks) / 2460)
            for i in range(1, 7):
                peaks, _ = find_peaks(resistance[window_intervals_odd[i - 1]:window_intervals_odd[i]])
                avg_minima_con[i + 6].append(len(peaks) / 2460)
                
                
            epsilon = np.finfo(float).eps
            if (sample_rate * 15 > len(resistance)):
                f = np.full(len(resistance), np.nan)
                P = np.full(len(resistance), np.nan)
            else:
                f, P = welch(resistance, sample_rate, nperseg = sample_rate * 15, noverlap = sample_rate * 10)
            #print(np.sum(P(f > 0 and f <= 0.1)))
            psd_subband_01.append(np.log(np.sum(P[(f > 0) & (f <= 0.1)]) + epsilon))
            psd_subband_02.append(np.log(np.sum(P[(f > 0.1) & (f <= 0.2)]) + epsilon))
            psd_subband_03.append(np.log(np.sum(P[(f > 0.2) & (f <= 0.3)]) + epsilon))
            psd_subband_04.append(np.log(np.sum(P[(f > 0.3) & (f <= 0.4)]) + epsilon))    
            
            
            std_skin_conductance[0].append(np.std(conductance[:window_intervals_even[0]]))
            for i in range(1, 6):
                std_skin_conductance[i].append(np.std(conductance[window_intervals_even[i - 1]:window_intervals_even[i]]))
            std_skin_conductance[6].append(np.std(conductance[window_intervals_even[5]:]))
            for i in range(1, 7):
                std_skin_conductance[i + 6].append(np.std(conductance[window_intervals_odd[i - 1]:window_intervals_odd[i]]))
            
            
            first_derivatives = np.diff(conductance[:window_intervals_even[0]]) / del_t
            mean_first_derivative_con[0].append(np.mean(first_derivatives))
            for i in range(1, 6):
                first_derivatives = np.diff(conductance[window_intervals_even[i - 1]:window_intervals_even[i]]) / del_t
                mean_first_derivative_con[i].append(np.mean(first_derivatives))
            first_derivatives = np.diff(conductance[window_intervals_even[5]:]) / del_t
            mean_first_derivative_con[6].append(np.mean(first_derivatives))
            for i in range(1, 7):
                first_derivatives = np.diff(conductance[window_intervals_odd[i - 1]:window_intervals_odd[i]]) / del_t
                mean_first_derivative_con[i + 6].append(np.mean(first_derivatives))
                
            
            first_derivatives = np.diff(conductance[:window_intervals_even[0]]) / del_t
            mean_abs_first_derivative_con[0].append(np.mean(np.absolute(first_derivatives)))
            for i in range(1, 6):
                first_derivatives = np.diff(conductance[window_intervals_even[i - 1]:window_intervals_even[i]]) / del_t
                mean_abs_first_derivative_con[i].append(np.mean(np.absolute(first_derivatives)))
            first_derivatives = np.diff(conductance[window_intervals_even[5]:]) / del_t
            mean_abs_first_derivative_con[6].append(np.mean(np.absolute(first_derivatives)))
            for i in range(1, 7):
                first_derivatives = np.diff(conductance[window_intervals_odd[i - 1]:window_intervals_odd[i]]) / del_t
                mean_abs_first_derivative_con[i + 6].append(np.mean(np.absolute(first_derivatives)))
                
                
            first_derivatives = np.diff(conductance[:window_intervals_even[0]]) / del_t
            second_derivatives = np.diff(first_derivatives) / del_t
            mean_abs_second_derivative_con[0].append(np.mean(np.absolute(second_derivatives)))
            for i in range(1, 6):
                first_derivatives = np.diff(conductance[window_intervals_even[i - 1]:window_intervals_even[i]]) / del_t
                second_derivatives = np.diff(first_derivatives) / del_t
                mean_abs_second_derivative_con[i].append(np.mean(np.absolute(second_derivatives)))
            first_derivatives = np.diff(conductance[window_intervals_even[5]:]) / del_t
            second_derivatives = np.diff(first_derivatives) / del_t
            mean_abs_second_derivative_con[6].append(np.mean(np.absolute(second_derivatives)))
            for i in range(1, 7):
                first_derivatives = np.diff(conductance[window_intervals_odd[i - 1]:window_intervals_odd[i]]) / del_t
                second_derivatives = np.diff(first_derivatives) / del_t
                mean_abs_second_derivative_con[i + 6].append(np.mean(np.absolute(second_derivatives)))
                
            
            peaks, _ = find_peaks(conductance[:window_intervals_even[0]])
            avg_minima_res[0].append(len(peaks) / 2460)
            for i in range(1, 6):
                peaks, _ = find_peaks(conductance[window_intervals_even[i - 1]:window_intervals_even[i]]) 
                avg_minima_res[i].append(len(peaks) / 2460)
            peaks, _ = find_peaks(conductance[window_intervals_even[5]:]) 
            avg_minima_res[6].append(len(peaks) / 2460)
            for i in range(1, 7):
                peaks, _ = find_peaks(conductance[window_intervals_odd[i - 1]:window_intervals_odd[i]])
                avg_minima_res[i + 6].append(len(peaks) / 2460)
                
                
            psd_subband_11.append(np.log(np.sum(P[(f > 0.0) & (f <= 0.24)]) + epsilon))
            psd_subband_12.append(np.log(np.sum(P[(f > 0.24) & (f <= 0.48)]) + epsilon))
            psd_subband_13.append(np.log(np.sum(P[(f > 0.48) & (f <= 0.72)]) + epsilon))
            psd_subband_14.append(np.log(np.sum(P[(f > 0.72) & (f <= 0.96)]) + epsilon))
            psd_subband_15.append(np.log(np.sum(P[(f > 0.96) & (f <= 1.2)]) + epsilon))
            psd_subband_16.append(np.log(np.sum(P[(f > 1.2) & (f <= 1.44)]) + epsilon))
            psd_subband_17.append(np.log(np.sum(P[(f > 1.44) & (f <= 1.68)]) + epsilon))
            psd_subband_18.append(np.log(np.sum(P[(f > 1.68) & (f <= 1.92)]) + epsilon))
            psd_subband_19.append(np.log(np.sum(P[(f > 1.92) & (f <= 2.16)]) + epsilon))
            psd_subband_10.append(np.log(np.sum(P[(f > 2.16) & (f <= 2.4)]) + epsilon))  
            
            
    user_data = pd.DataFrame({
        'presentation_id' : presentation_id,
        #'mean_skin_resistance' : mean_skin_resistance,
        #'mean_first_derivative_res' : mean_first_derivative_res,
        #'mean_abs_first_derivative_res' : mean_abs_first_derivative_res,
        #'mean_neg_first_derivative_res' : mean_neg_first_derivative_res,
        #'percentage_neg_first_derivative_res' : percentage_neg_first_derivative_res,
        #'std_skin_resistance' : std_skin_resistance,
        #'avg_minima_con' : avg_minima_con,
        #'std_skin_conductance' : std_skin_conductance,
        #'mean_first_derivative_con' : mean_first_derivative_con,
        #'mean_abs_first_derivative_con' : mean_abs_first_derivative_con,
        #'mean_abs_second_derivative_con' : mean_abs_second_derivative_con,
        #'avg_minima_res' : avg_minima_res,
        'psd_subband_01' : psd_subband_01,
        'psd_subband_02' : psd_subband_02,
        'psd_subband_03' : psd_subband_03,
        'psd_subband_04' : psd_subband_04,
        'psd_subband_11' : psd_subband_11,
        'psd_subband_12' : psd_subband_12,
        'psd_subband_13' : psd_subband_13,
        'psd_subband_14' : psd_subband_14,
        'psd_subband_15' : psd_subband_15,
        'psd_subband_16' : psd_subband_16,
        'psd_subband_17' : psd_subband_17,
        'psd_subband_18' : psd_subband_18,
        'psd_subband_19' : psd_subband_19,
        'psd_subband_10' : psd_subband_10
    })
    
    for i in range(13):
        user_data['mean_skin_resistance_' + str(i)] = mean_skin_resistance[i]
        user_data['mean_first_derivative_res' + str(i)] = mean_first_derivative_res[i]
        user_data['mean_abs_first_derivative_res' + str(i)] = mean_abs_first_derivative_res[i]
        user_data['mean_neg_first_derivative_res' + str(i)] = mean_neg_first_derivative_res[i]
        user_data['percentage_neg_first_derivative_res' + str(i)] = percentage_neg_first_derivative_res[i]
        user_data['std_skin_resistance' + str(i)] = std_skin_resistance[i]
        user_data['avg_minima_con' + str(i)] = avg_minima_con[i]
        user_data['std_skin_conductance' + str(i)] = std_skin_conductance[i]
        user_data['mean_first_derivative_con' + str(i)] = mean_first_derivative_con[i]
        user_data['mean_abs_first_derivative_con' + str(i)] = mean_abs_first_derivative_con[i]
        user_data['mean_abs_second_derivative_con' + str(i)] = mean_abs_second_derivative_con[i]
        user_data['avg_minima_res' + str(i)] = avg_minima_res[i]
               
    data_frames.append(user_data)
            

User ID: 1


C:\Users\Danny\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Danny\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


User ID: 2
User ID: 3
User ID: 4
User ID: 5
User ID: 6
User ID: 7
User ID: 8
User ID: 9
User ID: 10
User ID: 11
User ID: 12
User ID: 13
User ID: 14
User ID: 15
User ID: 16
User ID: 17
User ID: 18
User ID: 19
User ID: 20
User ID: 21
User ID: 22
User ID: 23
User ID: 24
User ID: 25
User ID: 26
User ID: 27
User ID: 28
User ID: 29
User ID: 30
User ID: 31
User ID: 32
User ID: 33
User ID: 34
User ID: 35
User ID: 36
User ID: 37
User ID: 38
User ID: 39
User ID: 40
User ID: 41
User ID: 42
User ID: 43
User ID: 44
User ID: 45
User ID: 46
User ID: 47
User ID: 48
User ID: 49
User ID: 50
User ID: 51
User ID: 52
User ID: 53
User ID: 54
User ID: 55
User ID: 56
User ID: 57
User ID: 58


In [18]:
GSR_Features_FixedWindow = pd.concat(data_frames,ignore_index=True)

In [20]:
GSR_Features_FixedWindow

,presentation_id,psd_subband_01,psd_subband_02,psd_subband_03,psd_subband_04,psd_subband_10,psd_subband_11,psd_subband_12,psd_subband_13,psd_subband_14,...,mean_abs_first_derivative_res12,mean_neg_first_derivative_res12,percentage_neg_first_derivative_res12,std_skin_resistance12,avg_minima_con12,std_skin_conductance12,mean_first_derivative_con12,mean_abs_first_derivative_con12,mean_abs_second_derivative_con12,avg_minima_res12
0,0101,8.190351,7.269246,4.848420,4.811748,1.770592,8.525450,5.660059,4.207316,3.828457,...,0.007938,-0.005429,-0.000894,4.754296e+01,0.000000,3.727016e-08,-4.818825e-12,6.228703e-12,1.421504e-12,0.000407
1,0102,9.334036,8.693753,4.947524,5.101455,1.990728,9.757434,5.857650,4.412020,4.043192,...,0.000000,NaN,0.000000,7.275958e-12,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
2,0103,9.779973,8.458787,6.101878,6.153736,3.112103,10.016482,6.963362,5.545675,5.168266,...,0.000000,NaN,0.000000,0.000000e+00,0.000000,3.388132e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
3,0104,9.401422,8.614598,6.201149,5.459607,2.307026,9.776626,6.673023,4.745293,4.364244,...,0.000000,NaN,0.000000,7.275958e-12,0.000000,3.388132e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
4,0105,9.668437,8.600344,5.372266,5.224823,2.043477,9.963837,6.106410,4.478724,4.100475,...,0.000000,NaN,0.000000,7.275958e-12,0.000000,6.776264e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
5,0106,7.810304,6.920835,4.244702,4.157904,1.091222,8.154513,5.025547,3.523981,3.148261,...,0.000000,NaN,0.000000,7.275958e-12,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
6,0107,8.494534,7.843084,5.863046,5.658505,2.501753,8.914092,6.574861,4.960691,4.575054,...,0.000000,NaN,0.000000,7.275958e-12,0.000000,6.776264e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
7,0108,15.998362,16.658153,15.485439,15.331758,11.748587,17.074861,16.179788,14.288631,13.840194,...,0.000000,NaN,0.000000,0.000000e+00,0.000000,3.388132e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
8,0109,7.668205,6.426387,3.634641,3.345632,0.050996,7.921962,4.291366,2.508986,2.126525,...,0.000000,NaN,0.000000,0.000000e+00,0.000000,6.776264e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
9,0110,9.811301,8.846112,5.783863,5.254465,1.894566,10.134045,6.327329,4.376454,3.975777,...,0.000000,NaN,0.000000,1.455192e-11,0.000000,3.388132e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000


In [22]:
GSR_Features_FixedWindow.to_excel(save_path)